needed files for run:

inverted_index_colab.py

part15_preprocessed.pkl

search_engine

search_frontend




In [1]:
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-9906  GCE       2                                       RUNNING  us-central1-a


In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
# !pip install pyspark

In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
from collections import defaultdict


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 12 14:35 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [8]:
spark

In [9]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'buck_3_final' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if 'multistream' in b.name:
        paths.append(full_path+b.name)

# paths = paths[:-1]

In [34]:
from pathlib import Path 
import os

# parquetFile = spark.read.parquet(*paths)
title_docs = parquetFile.select("title", "id").rdd.map(lambda e: (e['id'], e['title']))
# body_docs = parquetFile.select("id", "text").rdd.map(lambda r: (r[0],r[1]))
# anchor_docs=parquetFile.select("id", "anchor_text").rdd.map(lambda r: (r[0],r[1]))



In [39]:
# convert id2title into dict pickle
with open('id2title.pkl', 'wb') as f:
    pickle.dump(dict(title_docs.collect()),f)

!gsutil -m cp -r id2title.pkl gs://buck_3_final/ 

Copying file://id2title.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1/1 files][168.9 MiB/168.9 MiB] 100% Done                                    
Operation completed over 1 objects/168.9 MiB.                                    


### Define funcs to build the index

In [12]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
from inverted_index_gcp import InvertedIndex

In [13]:
%cd -q /home/dataproc
!mkdir title_index
!mkdir body_index
!mkdir anchor_index

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS


def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in 
    `all_stopwords` and return entries that will go into our posting lists. 
    Parameters:
    -----------
        text: str
        Text of one document
        id: int
        Document id
    Returns:
    --------
        List of tuples
        A list of (token, (doc_id, tf)) pairs 
        for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    counts = {}
    for t in tokens:
        if t not in all_stopwords:
            counts[t] = counts.get(t, 0)+1
    return [(w, (id,f)) for (w,f) in counts.items()]


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl, key=lambda e: e[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda e: (e[0], len(e[1])))

def partition_postings_and_write(postings, base_dir):
    ''' A function that partitions the posting lists into buckets, writes out 
    all posting lists in a bucket to disk, and returns the posting locations for 
    each bucket. Partitioning should be done through the use of `token2bucket` 
    above. Writing to disk should use the function  `write_a_posting_list`, a 
    static method implemented in inverted_index_colab.py under the InvertedIndex 
    class. 
    Parameters:
    -----------
        postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
        RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and 
        offsets its posting list was written to. See `write_a_posting_list` for 
        more details.
    '''

    postings_buckets = postings.map(lambda e: (token2bucket_id(e[0]),e))
    postings_buckets = postings_buckets.groupByKey()
    return postings_buckets.map(lambda e: InvertedIndex.write_a_posting_list(e, base_dir))

def merge_locs(posting_locs):
    super_posting_locs_title = defaultdict(list)
# for posting_loc in posting_locs_list_title:
#   for k, v in posting_loc.items():
#     super_posting_locs_title[k].extend(v)
    for def_dict in posting_locs:
        for term, locs in def_dict.items():
            super_posting_locs_title[term].extend(locs)
    return super_posting_locs_title

def calc_term_total(postings):
    s = 0
    print(postings)
    for x in postings[1]:
        s += x[1]
    return (postings[0], s)

def calc_DL(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]
    return (id, len(tokens))

    

mkdir: cannot create directory ‘title_index’: File exists
mkdir: cannot create directory ‘body_index’: File exists
mkdir: cannot create directory ‘anchor_index’: File exists


## build title index

In [19]:
title_docs_pls = title_docs.flatMap(lambda e: word_count(e[1], e[0]))
title_docs_pls = title_docs_pls.groupByKey().mapValues(reduce_word_counts)
title_locs = partition_postings_and_write(title_docs_pls, 'buck_3_final').collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

title_df = calculate_df(title_docs_pls).collectAsMap()
title_term_total = title_docs_pls.map(calc_term_total).collectAsMap()

title_index = InvertedIndex()
title_index.posting_locs = super_posting_locs
title_index.df = title_df
title_index.term_total = title_term_total
title_index.write_index('.', 'title_index')

index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 90.6 MiB/ 90.6 MiB]                                                
Operation completed over 1 objects/90.6 MiB.                                     


## build body index

In [21]:
body_docs_pls = body_docs.flatMap(lambda e: word_count(e[1], e[0]))
body_docs_pls = body_docs_pls.groupByKey().mapValues(reduce_word_counts)
body_locs = partition_postings_and_write(body_docs_pls, bucket_name).collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

dl_body = body_docs.map(lambda x:calc_DL(x[1],x[0])).collectAsMap()
body_df = calculate_df(body_docs_pls).collectAsMap()
body_term_total = body_docs_pls.map(calc_term_total).collectAsMap()

body_index = InvertedIndex()
body_index.posting_locs = super_posting_locs
body_index.df = body_df
body_index.DL = dl_body
body_index.term_total = body_term_total
body_index.write_index('.', 'body_index')

index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://body_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][  1.7 GiB/  1.7 GiB]  103.4 MiB/s                                   
Operation completed over 1 objects/1.7 GiB.                                      


## build anchor index

In [29]:
anchor=anchor_docs.flatMap(lambda x: x[1])
anchor=anchor.distinct().groupByKey().mapValues(lambda x: " ".join(x))
anchor_pls = anchor.flatMap(lambda x: word_count(x[1], x[0]))
anchor_pls = anchor_pls.groupByKey().mapValues(reduce_word_counts)
anchor_locs = partition_postings_and_write(anchor_pls,bucket_name).collect()


# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
dl_anchor=anchor.map(lambda x:calc_DL(x[1],x[0])).collectAsMap()
anchor_df = calculate_df(anchor_pls).collectAsMap()
anchor_term_total = anchor_pls.map(calc_term_total).collectAsMap()

anchor_index = InvertedIndex()
anchor_index.posting_locs = super_posting_locs
anchor_index.df = anchor_df
anchor_index.term_total = anchor_term_total
anchor_index.DL = dl_anchor
anchor_index.write_index('.', 'anchor_index')

index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][166.6 MiB/166.6 MiB]                                                
Operation completed over 1 objects/166.6 MiB.                                    


### move indices to their relevant directory

In [ ]:
!gsutil cp -m gs://buck_3_final/postings_gcp gs://buck_3_final/title_index
!gsutil -m cp -r gs://buck_3_final/postings_gcp gs://buck_3_final/anchor_index
!gsutil -m cp -r gs://buck_3_final/postings_gcp gs://buck_3_final/body_index

## download indices from storage

In [13]:
!gsutil -m cp -r gs://buck_3_final/title_index title_index
!gsutil -m cp -r gs://buck_3_final/anchor_index anchor_index
!gsutil -m cp -r gs://buck_3_final/body_index body_index

IndentationError: unexpected indent (4204836408.py, line 12)

## Get page Ranks

In [13]:
import numpy as np

def my_unzip(r):
    '''
        take take row of the format [(id0,[id1,id2...])], and unzip
        it to pairs, to the format [(id0,id1), (id0,id2), (id0,id3), ...] 
        this function is used as map function for the MapReduce
    '''
    return [(r[0],e) for e in r[1]]

def generate_graph(pages):
    ''' Compute the directed graph generated by wiki links.
    Parameters:
    -----------
        pages: RDD
        An RDD where each row consists of one wikipedia articles with 'id' and 
        'anchor_text'.
    Returns:
    --------
        edges: RDD
        An RDD where each row represents an edge in the directed graph created by
        the wikipedia links. The first entry should the source page id and the 
        second entry is the destination page id. No duplicates should be present. 
        vertices: RDD
        An RDD where each row represents a vetrix (node) in the directed graph 
        created by the wikipedia links. No duplicates should be present. 
    '''
    # define indices
    ID = 0
    ANCHOR_TEXT = 1
    pages = pages.map(lambda r: (r[ID], [int(x) for x in np.unique([i[ID] for i in r[ANCHOR_TEXT]])]))
    # return pages
    
    vertices = pages.map(lambda e: [e[0]]+e[1])
    vertices = vertices.flatMap(lambda e:e).distinct().map(lambda e:Row(id=e))
    pages = pages.map(my_unzip)
    
    edges = pages.flatMap(lambda e:e)

    return edges, vertices

In [15]:
edges, vertices = generate_graph(anchor_docs)
v_cnt, e_cnt = vertices.count(), edges.count()

edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv('pr', compression="gzip")
pr.show()

AnalysisException: path hdfs://cluster-9906-m/user/root/pr already exists.

In [ ]:
## move pageRank dict to cloud storage as pickle

In [ ]:
!hadoop fs -get hdfs://cluster-9906-m/user/root/pr page_rank
%cd -q page_rank
!gunzip pageRank.csv.gz


In [28]:
df = pd.read_csv('pageRank.csv', index_col=0, names=['rank']).to_dict()['rank']  # {doc_id:rank}
with open('pageRank.pkl', 'wb') as f:
    pickle.dump(df,f)
    

In [33]:
!gsutil -m cp -r pageRank.pkl gs://buck_3_final/ 

Copying file://pageRank.pkl [Content-Type=application/octet-stream]...
- [1/1 files][ 84.7 MiB/ 84.7 MiB] 100% Done                                    
Operation completed over 1 objects/84.7 MiB.                                     
